In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import linear_model

In [2]:
# https://www.kaggle.com/tunguz/deep-solar-dataset
df_all = pd.read_csv('cleaned_solar_tract_25.csv', encoding='latin-1')

In [3]:
for col in df_all.columns:
    try:
        pd.to_numeric(df_all[col])
    except:
        print(col)

# df = df_all#.dropna()

county
state
electricity_price_transportation


In [4]:
df_all = df_all.loc[:, ~(df_all.columns).isin(['voting_2012_dem_percentage', 'voting_2012_gop_percentage'])]

In [5]:
# 1.) pull out columns that contain strings, solar variables (except target), nonresidential variables

misc = ['Unnamed: 0','Unnamed: 0.1','fips', 'county', 'state']

solar = ['total_panel_area',  'total_panel_area_residential', 'total_panel_area_nonresidential',  
                   'solar_panel_area_divided_by_area', 'solar_panel_area_per_capita', 'heating_fuel_solar', 'heating_fuel_solar_rate',
                'tile_count','tile_count_residential', 'tile_count_nonresidential','number_of_solar_system_per_household', 
                   'solar_system_count','solar_system_count_nonresidential']
non_residential = ['electricity_price_transportation','incentive_count_nonresidential','incentive_nonresidential_state_level',
                   'electricity_consume_commercial','electricity_price_overall', 'electricity_consume_industrial', 
                   'electricity_consume_total','electricity_price_industrial','electricity_price_commercial']

df = df_all.loc[:, ~(df_all.columns).isin(misc + solar + non_residential)]

In [6]:
def split_scale_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    sclr_X = StandardScaler()
    sclr_X.fit(X_train[X_train.columns])
    
    sclr_y = StandardScaler()
    sclr_y.fit(y_train)

    X_train_scaled = pd.DataFrame(sclr_X.transform(X_train[X_train.columns]), columns = X_train.columns)
    X_test_scaled = pd.DataFrame(sclr_X.transform(X_test[X_test.columns]), columns = X_test.columns)

    y_train_scaled = sclr_y.transform(y_train)
    y_test_scaled = sclr_y.transform(y_test)
    
    return X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_train, X_test, y_train, y_test, sclr_X, sclr_y

# All Columns

In [7]:
target = 'solar_system_count_residential'

X = df.loc[:, ~(df.columns).isin([target])]
y = df[target]

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled, X_train, X_test, y_train, y_test, sclr_X, sclr_y = split_scale_data(X, y.reshape(-1, 1))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


## All columns: Non-PCA 

In [8]:
from sklearn import ensemble

# Fit regression model
params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

clf_nopca = ensemble.GradientBoostingRegressor(**params)

clf_nopca.fit(X_train, y_train)
test_score_nopca = clf_nopca.score(X_test, y_test)

print('test score:', test_score_nopca)

# test score: 0.721836427336
# test score: 0.759038089203 

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test score: 0.759038089203


In [ ]:
cv_scores_nopca = cross_val_score(clf_nopca, X_train, y_train, cv = 6)
print('cv_scores:', cv_scores_nopca)

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [33]:
# https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)

# #############################################################################
# Plot training deviance

# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(clf.staged_predict(X_test)):
    test_score[i] = clf.loss_(y_test, y_pred)

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')

# #############################################################################
# Plot feature importance
feature_importance = clf.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.subplot(1, 2, 2)
plt.barh(pos[:30], feature_importance[sorted_idx][:30], align='center')
plt.yticks(pos[:30], X_test.columns[sorted_idx][:30])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')

MSE: 768.2518


KeyboardInterrupt: 

## All Columns: PCA

In [9]:
pca_120 = PCA(n_components=120)
pca_120.fit(X_train_scaled)

PCA(copy=True, iterated_power='auto', n_components=120, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [ ]:
clf_pca = ensemble.GradientBoostingRegressor(**params)

clf_pca.fit(pca.transform(X_train_scaled), y_train_scaled)
test_score_pca = clf_pca.score(pca_120.transform(X_test_scaled), y_test_scaled)
print('test score:', test_score_pca)

# on 100 components: test score: 0.590027047835
# on 120 components: test score: 0.603272862949

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [52]:
cv_scores_pca = cross_val_score(clf_pca, pca.transform(X_train_scaled), y_train_scaled, cv = 6)
print('cv_scores:', cv_scores_pca)

# on 120 components: cv_scores: [ 0.62884166  0.61601805  0.6327424   0.62992872  0.60306967  0.63781009]

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

cv_scores: [ 0.62884166  0.61601805  0.6327424   0.62992872  0.60306967  0.63781009]


# Omit rate features

In [38]:
edu_omit = ['education_bachelor','education_college','education_doctoral','education_high_school_graduate','education_less_than_high_school','education_master','education_professional_school']
race_omit = ['race_asian','race_black_africa','race_indian_alaska','race_islander','race_other','race_two_more','race_white', 'race_indian_alaska_rate','race_two_more_rate']
heat_fuel_omit = ['heating_fuel_coal_coke','heating_fuel_electricity','heating_fuel_fuel_oil_kerosene','heating_fuel_gas','heating_fuel_housing_unit_count',
'heating_fuel_none','heating_fuel_other','heating_fuel_solar', 'heating_fuel_solar_rate']


edu_rate_omit = ['education_bachelor_rate','education_college_rate','education_doctoral_rate','education_high_school_graduate_rate','education_less_than_high_school_rate','education_master_rate','education_professional_school_rate']
race_rate_omit = ['race_asian_rate','race_black_africa_rate','race_indian_alaska_rate','race_islander_rate','race_other_rate','race_two_more_rate','race_white_rate','race_indian_alaska','race_two_more']
heat_fuel_rate_omit = ['heating_fuel_coal_coke_rate','heating_fuel_electricity_rate','heating_fuel_fuel_oil_kerosene_rate','heating_fuel_gas_rate','heating_fuel_housing_unit_count_rate',
'heating_fuel_none_rate','heating_fuel_other_rate','heating_fuel_solar_rate', 'heating_fuel_solar']


df_no_ratefeatures = df.loc[:, ~(df.columns).isin(edu_rate_omit + race_rate_omit + heat_fuel_rate_omit)]

In [ ]:
X_train_no_ratefeatures = X_train.loc[:, ~(X_train.columns).isin(edu_rate_omit + race_rate_omit + heat_fuel_rate_omit)]
X_test_no_ratefeatures = X_test.loc[:, ~(X_test.columns).isin(edu_rate_omit + race_rate_omit + heat_fuel_rate_omit)]
y_train_no_ratefeatures = y_train
y_test_no_ratefeatures = y_test

X_train_scaled_no_ratefeatures = X_train_scaled.loc[:, ~(X_train_scaled.columns).isin(edu_rate_omit + race_rate_omit + heat_fuel_rate_omit)]
X_test_scaled_no_ratefeatures = X_test_scaled.loc[:, ~(X_test_scaled.columns).isin(edu_rate_omit + race_rate_omit + heat_fuel_rate_omit)]
y_train_scaled_no_ratefeatures = y_train_scaled
y_test_scaled_no_ratefeatures = y_test_scaled

In [39]:
# target = 'solar_system_count_residential'

# X = df_no_ratefeatures.loc[:, ~(df_no_ratefeatures.columns).isin([target])]
# y = df_no_ratefeatures[target]

# X_train_scaled_no_ratefeatures, X_test_scaled_no_ratefeatures, y_train_scaled_no_ratefeatures, y_test_scaled_no_ratefeatures, X_train_no_ratefeatures, X_test_no_ratefeatures, y_train_no_ratefeatures, y_test_no_ratefeatures, sclr_X_no_ratefeatures, sclr_y_no_ratefeatures = split_scale_data(X, y.reshape(-1, 1))


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:11: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  # This is added back by InteractiveShellApp.init_path()


## Omit Columns: Non-PCA

In [47]:
# Fit regression model
params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.01, 'loss': 'ls'}

clf_no_ratefeatures = ensemble.GradientBoostingRegressor(**params)

clf_no_ratefeatures.fit(X_train_no_ratefeatures, y_train_no_ratefeatures)
test_score_nopca_noratefeatures = clf_no_ratefeatures.score(X_test_no_ratefeatures, y_test_no_ratefeatures)

print('test score:', test_score_nopca_noratefeatures)

# test score: 0.756760503454

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test score: 0.756760503454


In [ ]:
cv_scores_no_ratefeatures = cross_val_score(clf_no_ratefeatures, X_train_no_ratefeatures, y_train_no_ratefeatures, cv = 6)
print('cv_scores:', cv_scores_no_ratefeatures)

## Omit Columns: PCA

In [42]:
pca = PCA(n_components=100)
pca.fit(X_train_scaled_no_ratefeatures)

PCA(copy=True, iterated_power='auto', n_components=100, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [43]:
clf2_pca_no_ratefeatures = ensemble.GradientBoostingRegressor(**params)

clf2_pca_no_ratefeatures.fit(pca.transform(X_train_scaled_no_ratefeatures), y_train_scaled_no_ratefeatures)
test_score_pca_no_ratefeatures = clf2_pca_no_ratefeatures.score(pca.transform(X_test_scaled_no_ratefeatures), y_test_scaled_no_ratefeatures)
print('test score:', test_score_pca_no_ratefeatures)

# on 100 components: test score: 0.637466864954

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


test score: 0.637466864954


In [45]:
cv_scores_pca_no_ratefeatures = cross_val_score(clf2_pca_no_ratefeatures, pca.transform(X_train_scaled_no_ratefeatures), y_train_scaled_no_ratefeatures, cv = 6)
print('cv_scores:', cv_scores_pca_no_ratefeatures)

# on 100 components: cv_scores: [ 0.64649082  0.62461106  0.6024068   0.57162299  0.6133      0.61602934]

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

cv_scores: [ 0.64649082  0.62461106  0.6024068   0.57162299  0.6133      0.61602934]
